In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from time import sleep
from processor.processor import Processor as p
from extractor.tiingo_extractor import TiingoExtractor
from extractor.fred_extractor import FREDExtractor
from fund.fund import Fund
from pricer.pricer import Pricer as pricer_list
from ranker.ranker import Ranker as ranker_list
from classifier.classifier import Classifier as classifier_list
from portfolio.aportfolio import APortfolio

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
market.connect()
start = market.retrieve_max_date("stocks")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")

In [4]:
analysis = []
market.connect()
periods = ["year","quarter","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("stocks",ticker)
        test = p.column_date_processing(test)
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["rolling_20"] = test["adjclose"].rolling(window=20).mean()
        test["window_10"] = test["adjclose"].shift(10)
        ep = test.tail(1)["adjclose"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjclose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjclose"].item())/test.tail(2).iloc[0]["adjclose"].item()
        report["rolling_gain"] = (test.tail(2).iloc[0]["rolling_20"].item() - ep)/ep
        report["window_gain"] = (test.tail(2).iloc[0]["window_10"].item() - ep)/ep
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:31<00:00, 15.41it/s]


In [9]:
a = pd.DataFrame(analysis)
recs = a.sort_values("rolling_gain",ascending=False).head(10)
recs["date"] = end
recs

,year_gain,quarter_gain,week_gain,day_gain,window_gain,ticker,date
282,-0.681564,-0.240000,-0.065574,-0.055249,0.192982,LUMN,2023-07-28 05:33:43.924665
16,0.118121,-0.113743,-0.116070,-0.015044,0.187739,ALK,2023-07-28 05:33:43.924665
17,-0.038071,-0.101675,-0.023008,-0.019379,0.182034,ALB,2023-07-28 05:33:43.924665
411,0.023494,-0.108531,-0.112366,-0.089355,0.167474,LUV,2023-07-28 05:33:43.924665
244,-0.001427,-0.141207,0.025130,0.019506,0.156353,IPG,2023-07-28 05:33:43.924665
342,0.005636,-0.138342,-0.009026,-0.006875,0.156060,OMC,2023-07-28 05:33:43.924665
145,0.080013,-0.117792,-0.042815,-0.021692,0.143406,DFS,2023-07-28 05:33:43.924665
171,0.024901,-0.114651,-0.017865,-0.003145,0.142611,EFX,2023-07-28 05:33:43.924665
248,-0.104762,-0.057210,-0.100780,-0.017640,0.139319,IVZ,2023-07-28 05:33:43.924665
383,-0.150480,-0.134363,-0.122049,-0.013094,0.136785,RTX,2023-07-28 05:33:43.924665


In [6]:
portfolio = APortfolio(pricer_list.DAILY_STOCK_SPECULATION
                          ,classifier_list.NONE
                          ,ranker_list.NONE)
portfolio_ii = APortfolio(pricer_list.DAILY_STOCK_ROLLING
                           ,classifier_list.NONE
                          ,ranker_list.NONE)
portfolio_iii = APortfolio(pricer_list.DAILY_STOCK_WINDOW
                          ,classifier_list.NONE
                          ,ranker_list.NONE)

portfolios = []
# portfolios.append(portfolio)
# portfolios.append(portfolio_ii)
portfolios.append(portfolio_iii)

In [8]:
start = datetime(datetime.now().year,1,1)
end = datetime.now()
fund = Fund(portfolios,start,end,end)
market = Market()
fund.initialize_portfolios()

In [12]:
portfolio = portfolios[0]
portfolio.db.cloud_connect()
portfolio.db.drop("recs")
portfolio.db.store("recs",recs)
portfolio.db.disconnect()